# test

In [ ]:
import torch
from nas.supernet.supernet_yolov7 import YOLOSuperNet
from latency_predictor.arch_utils import *

supernet = YOLOSuperNet(cfg='./yaml/yolov7_dynamicsupernet.yml').to('cpu')
supernet.set_active_subnet([1,1,1,1,1,1,1,1]) # Maximum: [5,5,5,5,7,7,7,7]
subnet = supernet.get_active_subnet()

_, h_arch, save = split_backbone_head(subnet)

out_path = 'exported_models/head_1_1_1_1.pt'

# .pt 저장
head = skin_head(h_arch, save)
torch.save(head, out_path)

####################################
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

torch_model = torch.load(out_path)
input_tensor = torch.rand(1, 1024, 20, 20)
onnx_model_path = "exported_models/_test.onnx"
tf_model_path = "exported_models/_test.pb"
tflite_model_path = "exported_models/_test.tflite"

torch.onnx.export(
    model=torch_model,
    args=input_tensor,
    f=onnx_model_path,
    verbose=False,
    export_params=True,
    do_constant_folding=False,
    input_names=["input"],
    opset_version=12,
    output_names=["output"],
)

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS,  # enable TensorFlow ops.
]
tflite_model = converter.convert()
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)
    
interpret = tf.lite.Interpreter(tflite_model_path)


# backbone/head .pt to .tflite

In [ ]:
import os

from torch2tflite.converter import Torch2TFLiteConverter

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def arch2tflite(name: str, out_dir: str):

    base_path = os.path.join(out_dir, name)
    
    if 'backbone' in name:
        dummy_input_shape = (1, 3, 640, 640)
    else:
        dummy_input_shape = (1, 1024, 20, 20)

    conv = Torch2TFLiteConverter(
        base_path + '.pt',
        base_path + '.tflite',
        target_shape = dummy_input_shape,
    )
    conv.convert(remove_tmp_dir=True)
    

from tqdm.auto import tqdm

out_dir = 'exported_models'

file_list = [d for d in os.listdir(out_dir) if d.endswith('pt') and d.startswith('head')]

for torch_file in tqdm(file_list):
    arch2tflite(torch_file[:-3], out_dir)
    os.remove(os.path.join(out_dir, torch_file))